# Denver Crime Heatmap Notebook
CS 3120 Machine Learning Term Project - Konstantin Zaremski

---

This comprehensive notebook covers my project from ideation and planning, to exploratory data analysis (EDA), and final implementation.

> ### Project Proposal
> ##### Introduction & Description
> After my car was vandalized in Denver earlier this October, I began ruminating on what makes an area “safe”. Eventually I came upon the Denver Public Crime Map which shows the 1000 most recent incidents on a map. This got me interested in the possibility of using this data in a predictive capacity. I am going to build a Denver crime prediction or likelihood map app, that will predict the likelihood of crime overall, predict the most likely crimes to be committed for different areas, and then present this as a heatmap visualization to the user.
> 
> ##### Dataset
> The source dataset for the public crime map is provided by Denver and contains 394,475 data points sourced from the FBI’s NIBRS database. After stripping off useless columns such as database keys and precinct numbers, the usable columns are the date and time of occurrence, type and category of crime, and longitude and latitude. Crimes do not happen because of a longitude and latitude value, so I am enriching the original dataset by adding additional features by querying each data point against Open Street Map data for land use, proximity to residential or main roads, building types, and amenities such as ATMs or RTD stations.
> 
> ##### Model
> I will be training two models to make predictions for my app: one model to enable the prediction of the total amount of crimes in an area given input features, and another model to perform multiclass classification to predict what type of crime is most likely to be committed in an area given input features. Since the input data and resulting crime or number of crimes is known and labeled, supervised learning models are best suited for this data. I will be using SciKit Learn’s HistGradientBoostingClassifier to predict crime type and HistGradientBoostingRegressor to predict crime volume. I am electing to use gradient boosting over decision trees or other models since it has a lower bias and should be more sensitive to different crime patterns.
> 
> ##### App Functionality
> The web app will be a simple map interface with a color heatmap overlay based on the underlying model’s crime predictions. The user will have options for a time delta, where they can view the heatmap/predictions for the current time plus or minus up to 48 hours. The user will have a way to toggle between seeing a multi-color heatmap for the types of crimes that are predicted, a single-color heatmap for the likelihood or predicted frequency of crime, and a heatmap that combines both, with areas having a lower predicted likelihood or frequency of crime being denoted by a more transparent version of the color representing the crime type for that area.
> 
> ##### Technical Implementation
> Since the app does not have user accounts, the backend can be implemented as a simple Flask app with an API endpoint to retrieve the heatmap overlay based on the map area the client says the user is currently viewing. Predictions will be stored in an SQLite database where they can be quickly retrieved to generate a heatmap at view time. The frontend will be implemented using simple HTML, CSS, and vanilla JavaScript to enable interactions. The map itself will be rendered using the Leaflet JavaScript map library/project.
> 
> ##### Feasibility
> Since the app itself is a visualization with only a few buttons/interactions, and no user accounts or user data collection, it will be simple to implement, which will allow for more time to be spent on tuning the models and finding the optimal way to slice the map or bin data.
> 

## Exploratory Data Analysis & Data Preparation

### Setup
Importing required modules and libraries for EDA, etc. See `requirements_eda.txt` for a full list of requirements to install into the environment using `pip`.

In [1]:
from IPython.display import display
import pandas as pd
import math
import pyproj
import numpy as np
import glob

### Load Crime Data

In [2]:
# Find all crime data CSV files matching the naming pattern
csv_files = glob.glob("data/crime_split_*.csv")

# Read in all of the files and create Pandas dataframes from them
split_dfs = [pd.read_csv(f) for f in csv_files]

# Concatenate all the dataframes into a single one
df = pd.concat(split_dfs, ignore_index=True)

display(df.head())

original_dataframe_shape = df.shape
print(f'Loaded {original_dataframe_shape[0]} records!')

,OBJECTID,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,...,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,VICTIM_COUNT,x,y
0,20000,2020467360,2020467360299901,2999,1,criminal-mischief-mtr-veh,public-disorder,8/2/2020 10:43:00 PM,NaN,8/2/2020 10:43:00 PM,...,-105.024597,39.689751,4,422,ruby-hill,1,0,1,3.133789e+06,1.676471e+06
1,20001,20196003434,20196003434299901,2999,1,criminal-mischief-mtr-veh,public-disorder,4/20/2019 7:30:00 AM,4/20/2019 8:45:00 AM,4/20/2019 6:30:00 PM,...,-104.987348,39.714316,3,311,speer,1,0,1,3.144221e+06,1.685476e+06
2,20002,2020123049,2020123049299901,2999,1,criminal-mischief-mtr-veh,public-disorder,2/25/2020 11:30:00 PM,2/26/2020 6:45:00 AM,2/26/2020 7:30:00 AM,...,-104.988098,39.764528,6,612,five-points,1,0,1,3.143907e+06,1.703765e+06
3,20003,2021621685,2021621685299901,2999,1,criminal-mischief-mtr-veh,public-disorder,11/1/2021 6:20:00 AM,11/1/2021 6:30:00 AM,11/1/2021 10:15:00 AM,...,-105.004665,39.739669,1,123,lincoln-park,1,0,1,3.139299e+06,1.694684e+06
4,20004,2020289138,2020289138299901,2999,1,criminal-mischief-mtr-veh,public-disorder,5/9/2020 12:00:00 PM,NaN,5/11/2020 12:05:00 PM,...,-104.830661,39.795281,5,521,montbello,1,0,1,3.188083e+06,1.715255e+06


Loaded 394736 records!


### Column Analysis
The next thing to do is determine which columns will be useful for predictions and which columns need to be discarded.

In [3]:
print("Crime Dataset Columns:")
for column in df.columns:
    print(f" - {column}")

Crime Dataset Columns:
 - OBJECTID
 - INCIDENT_ID
 - OFFENSE_ID
 - OFFENSE_CODE
 - OFFENSE_CODE_EXTENSION
 - OFFENSE_TYPE_ID
 - OFFENSE_CATEGORY_ID
 - FIRST_OCCURRENCE_DATE
 - LAST_OCCURRENCE_DATE
 - REPORTED_DATE
 - INCIDENT_ADDRESS
 - GEO_X
 - GEO_Y
 - GEO_LON
 - GEO_LAT
 - DISTRICT_ID
 - PRECINCT_ID
 - NEIGHBORHOOD_ID
 - IS_CRIME
 - IS_TRAFFIC
 - VICTIM_COUNT
 - x
 - y


#### Column Decisions
| Column ID                 | Decision  | Explanation                                                                                        |
|---------------------------|-----------|----------------------------------------------------------------------------------------------------|
| `OBJECTID`                | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `INCIDENT_ID`             | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `OFFENSE_ID`              | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `OFFENSE_CODE`            | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `OFFENSE_CODE_EXTENSION`  | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `OFFENSE_TYPE_ID`         | Kept      | This is part of the output (y-values). We will keep this column and it will be the subject of prediction by the classification model. |
| `OFFENSE_CATEGORY_ID`     | Discarded | This is part of the output (y-values). This will be discarded in  |
| `FIRST_OCCURRENCE_DATE`   | Kept      | Timestamp of the event, this will be kept and used for predictions made based on the time of day. |
| `LAST_OCCURRENCE_DATE`    | Discarded | Undefined for many rows. We will discard this timestamp and rely on `FIRST_OCCURRENCE_DATE` instead.                              |
| `REPORTED_DATE`           | Discarded | This is part of the output (y-values). Report time delta from occurrence varies incident to incident. No predictive value.                             |
| `INCIDENT_ADDRESS`        | Discarded | Although address is useful, this column is not standardized with entries like "2400 block" rather than exact addresses in some cases. |
| `GEO_X`                   | Discarded | Related to the mapping platform the data was posted on. No predictive value.                    |
| `GEO_Y`                   | Discarded | Related to the mapping platform the data was posted on. No predictive value.                    |
| `GEO_LON`                 | Kept      | Location of the crime, this will be kept and used to make predictions based on location. |
| `GEO_LAT`                 | Kept      | Location of the crime, this will be kept and used to make predictions based on location. |
| `DISTRICT_ID`             | Discarded | This is part of the output (y-values) and we have no way of querying. No predictive value.                    |
| `PRECINCT_ID`             | Discarded | This is part of the output (y-values) and we have no way of querying. No predictive value.                             |
| `NEIGHBORHOOD_ID`         | Discarded | Although neighborhood is useful, as some are "rougher" than others, we don't have a standard way of querying.                             |
| `IS_CRIME`                | Discarded | This is part of the output (y-values). No predictive value.                             |
| `IS_TRAFFIC`              | Discarded | This is part of the output (y-values). No predictive value.                             |
| `VICTIM_COUNT`            | Discarded | This is part of the output (y-values). No predictive value.                             |
| `x`                       | Discarded | Related to the mapping platform the data was posted on. No predictive value.                     |
| `y`                       | Discarded | Related to the mapping platform the data was posted on. No predictive value.                    |

In [4]:
very_first_dataframe = df.copy()

# Keep only the columns that we want
df = df[['OFFENSE_TYPE_ID','FIRST_OCCURRENCE_DATE','GEO_LON','GEO_LAT']]

# Displaying a random sample of the data to confirm what it looks like
display(df.sample(n=20))

,OFFENSE_TYPE_ID,FIRST_OCCURRENCE_DATE,GEO_LON,GEO_LAT
113081,police-false-information,2/25/2020 1:23:00 PM,-104.961788,39.682689
180334,theft-shoplift,3/23/2019 2:00:00 PM,-104.939123,39.667547
272876,theft-of-motor-vehicle,12/8/2019 8:00:00 PM,-104.965060,39.754979
12242,theft-items-from-vehicle,6/6/2022 7:00:00 AM,-104.676993,39.844158
91358,theft-of-motor-vehicle,4/5/2023 6:30:00 AM,-104.991513,39.747421
109103,burglary-residence-by-force,4/12/2020 12:00:00 PM,-104.974962,39.735996
199263,theft-of-motor-vehicle,6/1/2022 11:35:00 PM,-105.001101,39.732382
158894,false-imprisonment,6/10/2021 7:30:00 PM,-104.832142,39.797782
178178,burglary-business-no-force,4/18/2020 12:43:00 AM,-104.980569,39.688525
207203,theft-items-from-vehicle,7/11/2024 8:00:00 AM,-104.991195,39.752503


### Time Binning
In this analysis, time binning is used to capture the cyclical and seasonal patterns inherent in criminal behavior. Crime often follows temporal trends—certain types of incidents may be more likely to occur at specific times of day, days of the week, or during particular seasons. For example, incidents related to nightlife might peak during late evening hours, while other crimes may correlate with daily commuter traffic or specific days like weekends. By extracting temporal features like `YEAR`, `DAY_OF_YEAR`, `DAY_OF_WEEK`, and a continuous `TIME` (floating-point hour), we enable the model to identify these patterns and trends. This granular binning enhances the predictive power of the model, allowing it to detect not only broad time-based patterns but also more subtle nuances in how crime evolves throughout the day, week, and year.

#### Temporal Features Breakdown
* `YEAR`: Provides a clear yearly context, which can be crucial for identifying long-term trends or annual shifts in patterns.
* `TIME` (floating point): Offers a precise measure of the time of day, allowing models to pick up on subtle temporal shifts within a 24-hour cycle.
* `DAY_OF_YEAR`: Captures the day within the year, useful for identifying seasonal patterns or trends that recur annually.
* `DAY_OF_WEEK`: Encodes weekly cycles, helping the model detect patterns associated with specific weekdays.

In [5]:
# Convert FIRST_OCCURRENCE_DATE to Pandas date time format
df['FIRST_OCCURRENCE_DATE'] = pd.to_datetime(df['FIRST_OCCURRENCE_DATE'], format='%m/%d/%Y %I:%M:%S %p')

# Extract date and time components
df['YEAR'] = df['FIRST_OCCURRENCE_DATE'].dt.year
# df['MONTH'] = df['FIRST_OCCURRENCE_DATE'].dt.month -- Removed in favor of DAY_OF_YEAR
# df['DAY_OF_MONTH'] = df['FIRST_OCCURRENCE_DATE'].dt.day -- Removed in favor of DAY_OF_YEAR
# df['TIME'] = (
#     df['FIRST_OCCURRENCE_DATE'].dt.hour + 
#     df['FIRST_OCCURRENCE_DATE'].dt.minute / 60 + 
#     df['FIRST_OCCURRENCE_DATE'].dt.second / 3600
# ) -- No longer doing floating point hour
# df['HOUR'] = df['FIRST_OCCURRENCE_DATE'].dt.hour
df['HOUR_COS'] = np.cos(2 * np.pi * df['FIRST_OCCURRENCE_DATE'].dt.hour / 24) # Learned that some models interpret hour 0 and 
df['HOUR_SIN'] = np.sin(2 * np.pi * df['FIRST_OCCURRENCE_DATE'].dt.hour / 24) #   hour 23 as far apart even though they are together. Sin/Cos make continuous.
df['DAY_OF_YEAR'] = df['FIRST_OCCURRENCE_DATE'].dt.dayofyear
df['DAY_OF_WEEK'] = df['FIRST_OCCURRENCE_DATE'].dt.dayofweek  # 0 = Monday, 6 = Sunday

# Finally, dropping the FIRST_OCCURRENCE_DATE column
df.drop(columns=['FIRST_OCCURRENCE_DATE'], inplace=True)

# Displaying a random sample of the data to confirm what it looks like
display(df.sample(n=5))

,OFFENSE_TYPE_ID,GEO_LON,GEO_LAT,YEAR,HOUR_COS,HOUR_SIN,DAY_OF_YEAR,DAY_OF_WEEK
339722,fraud-by-telephone,-104.887304,39.747042,2022,8.660254e-01,0.500000,68,2
310581,theft-parts-from-vehicle,-104.927623,39.675576,2022,-2.588190e-01,-0.965926,330,5
323035,theft-bicycle,-104.977265,39.731304,2020,-9.659258e-01,0.258819,305,5
355732,criminal-mischief-mtr-veh,-104.998509,39.732305,2022,-1.836970e-16,-1.000000,357,4
51393,theft-items-from-vehicle,-105.038749,39.686763,2020,-1.836970e-16,-1.000000,170,3


### Location Slicing & Binning 
In this step, we convert latitude and longitude into a 15-meter by 15-meter grid system using a projected coordinate system (UTM). This method ensures that each crime location is assigned to a unique, precise grid cell, facilitating accurate spatial analysis.

See <https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system>

#### Why This is Better Than Using Raw Latitude and Longitude

Using UTM coordinates overcomes the limitations of raw latitude and longitude, which vary in distance depending on location. UTM coordinates provide uniform distance measurements in meters, improving precision and making it easier to define consistent grid cells. This approach also simplifies spatial analysis by grouping data into fixed-size cells, making patterns like crime hotspots easier to identify and computationally more efficient to process.

In [6]:
GRID_SIZE = 15 # 15m x 15m grid cells

# Initialize UTM projection (assuming UTM Zone 13N for Denver)
#   https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system
#   https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system#/media/File:Universal_Transverse_Mercator_zones.svg
utm_proj = pyproj.Proj(proj="utm", zone=13, datum="WGS84")

def get_grid_block(lat, lon):
    try:
        # Convert latitude and longitude to UTM coordinates (X, Y)
        x, y = utm_proj(lon, lat)  # Correct order: lon, lat
        
        # Calculate the X and Y blocks
        x_block = int(x // GRID_SIZE)
        y_block = int(y // GRID_SIZE)
        
        return x_block, y_block
    except:
        # If there is an issue parsing, return -1
        return -1, -1

# Apply the function to the DataFrame
df[['X_BLOCK', 'Y_BLOCK']] = df.apply(lambda row: get_grid_block(row['GEO_LAT'], row['GEO_LON']), axis=1, result_type='expand')

# Drop rows where X_BLOCK or Y_BLOCK is -1
df = df[(df['X_BLOCK'] != -1) & (df['Y_BLOCK'] != -1)]

# Note change in size
post_null_geo_block_removal_dataframe_shape = df.shape
print(f'Dataframe size before removal: {original_dataframe_shape[0]} items')
print(f'Dataframe size after removal: {post_null_geo_block_removal_dataframe_shape[0]} items')
print(f'--> {original_dataframe_shape[0] - post_null_geo_block_removal_dataframe_shape[0]} items removed!')

# Finally, dropping the GEO_LAT and GEO_LON columns since we have replaced them with X_BLOCK and Y_BLOCK
df.drop(columns=['GEO_LAT'], inplace=True)
df.drop(columns=['GEO_LON'], inplace=True)

# Displaying a random sample of the data to confirm what it looks like
display(df.sample(n=5))

Dataframe size before removal: 394736 items
Dataframe size after removal: 394475 items
--> 261 items removed!


,OFFENSE_TYPE_ID,YEAR,HOUR_COS,HOUR_SIN,DAY_OF_YEAR,DAY_OF_WEEK,X_BLOCK,Y_BLOCK
367367,assault-simple,2023,-9.659258e-01,-0.258819,292,3,33404,293445
315042,theft-other,2019,1.000000e+00,0.000000,147,0,33030,293210
277464,theft-shoplift,2024,-8.660254e-01,0.500000,93,1,33676,292700
328868,theft-other,2020,6.123234e-17,1.000000,27,0,35192,294091
211997,criminal-trespassing,2024,-5.000000e-01,0.866025,220,2,33446,293255


In this section, we are processing the geographical data to assign each crime incident to a specific grid cell based on its latitude and longitude. We achieve this by converting the coordinates to UTM projection, which provides more accurate distance-based calculations than traditional latitude and longitude. For each point, we calculate the corresponding grid cell by dividing the UTM coordinates by the grid size (15 meters) and then storing the results as `X_BLOCK` and `Y_BLOCK`.

However, if any coordinates cannot be processed correctly (due to projection issues or invalid data), we assign -1 as a placeholder. After applying this transformation, we remove any rows where either the `X_BLOCK` or `Y_BLOCK` is -1, indicating invalid grid assignments. The sizes of the DataFrame before and after this removal are printed to highlight the impact of cleaning the data. This step ensures that only valid data points are included for further analysis and modeling.

### Removing Location Outliers
When exporting the data as a CSV file, it can be observed that some of the rows have really low or high X-block values. These are likely location mis-inputs.

We can drop these rows by calculating the interquartile range range for the `X_BLOCK` and `Y_BLOCK` columns and dropping the rows that are outliers.

What I have done below is not the interquartile range. I only want to drop extreme outliers, so I dropped the values that fall outside the middle 98% of rows.

In [7]:
df_shape_before_location_outlier_drop = df.shape

Q1 = df['X_BLOCK'].quantile(0.01)
Q3 = df['X_BLOCK'].quantile(0.99)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df.loc[(df['X_BLOCK'] >= lower_bound) & (df['X_BLOCK'] <= upper_bound)]

# Calculate and display rows removed
df_shape_after_location_outlier_drop = df.shape
print(f'Size before location outlier removal: {df_shape_before_location_outlier_drop[0]} records.')
print(f'Size after location outlier removal: {df_shape_after_location_outlier_drop[0]} records.')
print(f'--> {df_shape_before_location_outlier_drop[0] - df_shape_after_location_outlier_drop[0]} records removed!')


Size before location outlier removal: 394475 records.
Size after location outlier removal: 394434 records.
--> 41 records removed!


### Adding in OpenStreetMap Features
In this section, we aim to enrich our dataset with spatial features from OpenStreetMap (OSM). By querying OSM data for geographic features based on the centroids of the X and Y grid blocks (calculated from latitude and longitude), we can incorporate valuable context such as the proximity to roads, points of interest, or land use types. This spatial data enhances our analysis by providing a richer understanding of the environment in which the crimes occur, potentially improving predictive modeling. By leveraging OpenStreetMap data, we can gather additional information about the geographical characteristics of each grid cell, allowing for more nuanced predictions based on spatial context.

In [8]:
# Imports
import sqlite3
from tqdm import tqdm
import json
import re
# Import osmium to parse the OSM files
import osmium

# Re-define the UTM projections from earlier
utm_proj = pyproj.Proj(proj="utm", zone=13, datum="WGS84")
latlon_proj = pyproj.Proj(proj="latlong", datum="WGS84")

# Define tranformers
latlon_to_utm_transformer = pyproj.Transformer.from_proj(latlon_proj, utm_proj)
utm_to_latlon_transformer = pyproj.Transformer.from_proj(utm_proj, latlon_proj)

'''
    Generate a Grid of Map Blocks/Cells from predefined corners.
'''
def gen_blocks_from_coordinates(southwest_corner, northeast_corner, database_path):
    # Bounding corners
    def dec_coordinates(str_coordinates):
        parts = str_coordinates.split(",")
        
        # Extract latitude from the first part and longitude from the second part
        lat = float(re.findall(r"\d+\.\d+", parts[0])[0])
        lon = float(re.findall(r"\d+\.\d+", parts[1])[0])
        
        # Adjust sign based on N/S and E/W indicators
        if "S" in parts[0]:
            lat = -lat
        if "W" in parts[1]:
            lon = -lon
            
        return lon, lat  # Return (longitude, latitude)

    southwest_lon, southwest_lat = dec_coordinates(southwest_corner)
    northeast_lon, northeast_lat = dec_coordinates(northeast_corner)

    # Define block size in meters (15 meters as mentioned)
    block_size = 15

    # Create a transformer for converting lat/lon to UTM
    latlon_to_utm_transformer = pyproj.Transformer.from_proj(latlon_proj, utm_proj)

    # Convert southwest and northeast corners to UTM
    southwest_x, southwest_y = latlon_to_utm_transformer.transform(southwest_lon, southwest_lat)
    northeast_x, northeast_y = latlon_to_utm_transformer.transform(northeast_lon, northeast_lat)

    # Calculate the area width and height in meters
    area_width = northeast_x - southwest_x
    area_height = northeast_y - southwest_y

    # Calculate the number of blocks in the area
    num_x_blocks = int(area_width // block_size)
    num_y_blocks = int(area_height // block_size)

    # Create SQLite database and table
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Create a table to store the map cells
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS blocks (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            x_block INTEGER,
            y_block INTEGER,
            cent_lat REAL,
            cent_lon REAL
        )
    ''')

    # Function to calculate the centroid of a block (in UTM)
    def get_centroid(x_block, y_block, grid_size=15):
        # Create a transformer for converting UTM to lat/lon
        utm_to_latlon_transformer = pyproj.Transformer.from_proj(utm_proj, latlon_proj)
        
        # Get the bottom-left corner coordinates in UTM
        x_origin, y_origin = x_block * grid_size, y_block * grid_size
        
        # Calculate the centroid by moving half the grid size in both directions
        x_centroid = x_origin + grid_size / 2
        y_centroid = y_origin + grid_size / 2
        
        # Convert UTM coordinates to latitude and longitude
        lon, lat = utm_to_latlon_transformer.transform(x_centroid, y_centroid)
        
        return lat, lon

    # Initialize the progress bar
    total_blocks = num_x_blocks * num_y_blocks

    # Set the batch size
    batch_size = 100000
    batch_data = []

    # Init the progress bar
    progress_bar = tqdm(total=total_blocks, desc="Identifying Block Centroids", position=0, ncols=120)

    # Loop through each block and calculate the centroid
    for y_block in range(num_y_blocks):
        for x_block in range(num_x_blocks):
            # Calculate the centroid for the current block
            lat, lon = get_centroid(x_block, y_block, block_size)
            
            # Append data to batch
            batch_data.append((x_block, y_block, lat, lon))
            
            # If batch size is reached, execute the batch insert
            if len(batch_data) >= batch_size:
                cursor.executemany('''
                    INSERT INTO blocks (x_block, y_block, cent_lat, cent_lon)
                    VALUES (?, ?, ?, ?)
                ''', batch_data)
                
                # Manually update the progress bar after each block
                progress_bar.update(len(batch_data))
                
                conn.commit()  # Commit after each batch
                batch_data = []  # Clear the batch data

    # Insert any remaining data after the loop
    if batch_data:
        cursor.executemany('''
            INSERT INTO blocks (x_block, y_block, cent_lat, cent_lon)
            VALUES (?, ?, ?, ?)
        ''', batch_data)

        progress_bar.update(len(batch_data))

        conn.commit()  # Final commit for remaining data

    # Close the progress bar once finished
    progress_bar.close()

    # Close the SQLite connection
    conn.close()

'''
    Generate a map based on the boundaries of the available crime location data, plus a margin
'''
def gen_map_from_crime_location_data_with_margin(crime_df, margin, database_path):
    # Get the limits of the locations saved within the data frame
    x_max = crime_df['X_BLOCK'].max()
    x_min = crime_df['X_BLOCK'].min()
    y_max = crime_df['Y_BLOCK'].max()
    y_min = crime_df['Y_BLOCK'].min()

    # Calculate the limits with the 
    x_max_margin = x_max + margin
    x_min_margin = x_min - margin
    y_max_margin = y_max + margin
    y_min_margin = y_min - margin

    # Print the map size
    print(f"Map Size Before Margin (in blocks): Upper Left Corner: ({x_min}, {y_max}), Lower Right Corner: ({x_max}, {y_min})")
    print(f"Map Size After Margin (in blocks): Upper Left Corner: ({x_min_margin}, {y_max_margin}), Lower Right Corner: ({x_max_margin}, {y_min_margin})")

    return

    # Function to get the edges of the box in lon/lat
    def block_to_latlon(x, y):
        # Project from UTM to latlon
        transformer = pyproj.Transformer.from_crs(f"epsg:32613", "epsg:4326", always_xy=True)
        return transformer.transform(x * GRID_SIZE, y * GRID_SIZE)

    min_longitude, min_latitude = block_to_latlon(x_min_margin, y_min_margin)
    max_longitude, max_latitude = block_to_latlon(x_max_margin, y_max_margin)
    print(f"    (lon, lat): ({min_longitude},{min_latitude}) ({max_longitude},{max_latitude})")

    # Loop through the each possible block in the map, including the margins
    total_blocks = (x_max_margin - x_min_margin) * (y_max_margin - y_min_margin)
    print(f'--> Total map grid cells: {total_blocks}')
    
    # Create SQLite database and table
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Create a table to store the map cells
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS blocks (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            x INTEGER,
            y INTEGER,
            c_lat REAL,
            c_lon REAL,
            tags TEXT
        )
    ''')

    # Set the batch size
    batch_size = 1
    batch_data = []
    
    # Init the progress bar
    progress_bar = tqdm(total=total_blocks, desc="Building Map", position=0, ncols=120)

    for x in range(x_min_margin, x_max_margin + 1):
        for y in range(y_min_margin, y_max_margin + 1):
            # Get important latitudes and longitudes
            center_lon, center_lat = block_to_latlon(x + (GRID_SIZE / 2), y - (GRID_SIZE / 2))

            tags = []
            
            for obj in osmium.FileProcessor('../denver_filtered.osm.pbf', osmium.osm.NODE):
                if obj.tags:
                    tags += obj,tags
                try:
                    if osmium.geom.haversine_distance(osmium.osm.Location(center_lon, center_lat), obj.location) < (GRID_SIZE * 1.5):
                        continue
                except:
                    print(obj)
            
            print(tags)
            
            batch_data.append((
                x,          #
                y,          #
                center_lon, #
                center_lat, #
                str(tags)        #
            ))

            # If batch size is reached, execute the batch insert
            if len(batch_data) >= batch_size:
                cursor.executemany('''
                    INSERT INTO blocks (x, y, c_lat, c_lon, tags)
                    VALUES (?, ?, ?, ?, ?)
                ''', batch_data)
                
                # Manually update the progress bar after each block
                progress_bar.update(len(batch_data))
                
                conn.commit()  # Commit after each batch
                batch_data = []  # Clear the batch data
            
    # Insert any remaining data after the loop
    if batch_data:
        cursor.executemany('''
            INSERT INTO blocks (x, y, c_lat, c_lon, tags)
            VALUES (?, ?, ?, ?, ?)
        ''', batch_data)

        progress_bar.update(len(batch_data))

        conn.commit()  # Final commit for remaining data

    # Close the SQLite connection
    conn.close()    
    
    # Close the progress bar once finished
    progress_bar.close()

# Old method was just a fixed area
southwest_corner = "39.53440° N, 105.24364° W"
northeast_corner = "40.03841° N, 104.58877° W"
database_path = 'map_grid.db'
#gen_blocks_from_coordinates(southwest_corner, northeast_corner, database_path)

gen_map_from_crime_location_data_with_margin(df, 400, 'map.db')
# -- Skipping... see below.

Map Size Before Margin (in blocks): Upper Left Corner: (32154, 294550), Lower Right Corner: (35553, 292331)
Map Size After Margin (in blocks): Upper Left Corner: (31754, 294950), Lower Right Corner: (35953, 291931)


$$
12676781 * 17 / 60 / 60 / 24 / 365 = 6.8336275051 \text{ years to process the OSM data}
$$
For now, I will set aside the extraction of OpenStreetMap (OSM) features due to the excessive processing time — approximately 6.83 years — required to handle the data with my current approach. The complexity of querying and filtering OSM data, even with optimization attempts, has proven to be a significant bottleneck. As a result, I will proceed with exploratory data analysis (EDA) using the existing crime dataset without incorporating OSM features. This will allow me to focus on other aspects of the analysis and revisit the OSM integration if a more efficient method becomes available.

## Early Model Training

In [9]:
# Save a copy of the initial dataframe
original_df = df.copy()

### Crime Type Model (Gradient Boosting Classifier)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, mean_squared_error

# Get a copy of the initial data frame
df = original_df.copy()

# Encode the target variable
encoder = LabelEncoder()
df['OFFENSE_TYPE_ID'] = encoder.fit_transform(df['OFFENSE_TYPE_ID'])

# Features and target
X = df.drop('OFFENSE_TYPE_ID', axis=1)
y = df['OFFENSE_TYPE_ID']

# Remove rare classes with only 1 instance
class_counts = y.value_counts()
rare_classes = class_counts[class_counts < 2].index

# Filter out the rare classes from the dataset
filtered_df = df[~df['OFFENSE_TYPE_ID'].isin(rare_classes)]

# Update X and y after removing rare classes
X = filtered_df.drop('OFFENSE_TYPE_ID', axis=1)
y = filtered_df['OFFENSE_TYPE_ID']

# Train/test split with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Ensure no class is left with fewer than 2 members in either train or test
train_class_counts = y_train.value_counts()
test_class_counts = y_test.value_counts()

# Print out the counts for train and test to debug
print("Train Class Distribution:\n", train_class_counts)
print("Test Class Distribution:\n", test_class_counts)

# Check if any class is left with 1 sample in the train set
if any(train_class_counts < 2):
    print("Some classes in the train set have less than 2 samples. Consider further reducing or combining classes.")
else:
    # Initialize and train the model
    model = HistGradientBoostingClassifier()
    model.fit(X_train, y_train)

    # Predictions
    y_pred = model.predict(X_test)

    # Accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    # Classification report (includes precision, recall, F1-score)
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

    # Precision, Recall, and F1 Score
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    print(f"Precision (weighted): {precision}")
    print(f"Recall (weighted): {recall}")
    print(f"F1 Score (weighted): {f1}")

    # RMSE (Root Mean Squared Error)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print(f"Root Mean Squared Error (RMSE): {rmse}")

    # To decode the predictions back to the original labels
    decoded_predictions = encoder.inverse_transform(y_pred)


Train Class Distribution:
 OFFENSE_TYPE_ID
150    47402
148    32539
35     22177
154    20602
153    18918
       ...  
92         2
177        2
68         2
90         2
86         2
Name: count, Length: 171, dtype: int64
Test Class Distribution:
 OFFENSE_TYPE_ID
150    11851
148     8135
35      5544
154     5151
153     4730
       ...  
113        1
86         1
123        1
109        1
94         1
Name: count, Length: 164, dtype: int64
Accuracy: 0.1315062241715894

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.03      0.05      0.04        61
           2       0.00      0.00      0.00       187
           3       0.10      0.00      0.01      1442
           4       0.07      0.00      0.00       656
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00        28
           7       0.00      0.00      0.00         1
          

/Users/kzaremski/Library/Mobile Documents/com~apple~CloudDocs/School/CS 3120 001 - Machine Learning/CS 3120 Project/denver-crime-heatmap/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kzaremski/Library/Mobile Documents/com~apple~CloudDocs/School/CS 3120 001 - Machine Learning/CS 3120 Project/denver-crime-heatmap/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kzaremski/Library/Mobile Documents/com~apple~CloudDocs/School/CS 3120 001 - Machine Learning/CS 3120 Project/de

### Crime Likelihood Model (Gradient Boosting Regressor)

In [11]:
# Get a copy of the initial data frame
df = original_df.copy()

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Group by grid cell, day of year, and hour to count crimes
crime_counts = df.groupby(['X_BLOCK', 'Y_BLOCK', 'DAY_OF_YEAR', 'HOUR_SIN', 'HOUR_COS']).size().reset_index(name='CRIME_COUNT')

# Features and target
X = crime_counts[['X_BLOCK', 'Y_BLOCK', 'DAY_OF_YEAR', 'HOUR_SIN', 'HOUR_COS']]
y = crime_counts['CRIME_COUNT']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = HistGradientBoostingRegressor()

# Train the model
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R2 Score: {r2}")


Mean Squared Error (MSE): 0.12031554289457475
Root Mean Squared Error (RMSE): 0.34686530944240407
Mean Absolute Error (MAE): 0.1601346201692215
R2 Score: 0.05012391604887978


### Early Conclusions & Evaluations of These Models
The regression model doesn't perform very well, with a low R² score of 0.052, meaning it doesn’t explain much of the variation in crime counts. The Mean Squared Error and Root Mean Squared Error suggest there's still a lot of room for improvement, with predictions off by about 0.35 crimes on average. The classification model also struggles, with low precision, recall, and F1 scores, pointing to issues with class imbalance and poor overall prediction accuracy. Although the accuracy is a bit higher, it doesn't capture the full picture of how well the model is performing across all classes. Both models need better feature engineering and more work on handling imbalances to improve their results.


## Exploring Alternatives: Neural Networks
Given the poor performance of my initial models, I am now exploring neural networks, including deep neural networks. With a limited number of features available and the infeasibility of incorporating OSM data, I believe that more complex models like these could help improve predictions. Neural networks are well-suited to handle intricate patterns in data and may offer better performance compared to the models I've tried so far. This approach seems like a promising next step to tackle the issues with accuracy and model generalization.

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Get a copy of the initial data frame
df = original_df.copy()

# Group by grid cell, day of year, and hour to count crimes
crime_counts = df.groupby(['X_BLOCK', 'Y_BLOCK', 'DAY_OF_YEAR', 'HOUR_SIN', 'HOUR_COS']).size().reset_index(name='CRIME_COUNT')

# Features and target
X = crime_counts[['X_BLOCK', 'Y_BLOCK', 'DAY_OF_YEAR', 'HOUR_SIN', 'HOUR_COS']]
y = crime_counts['CRIME_COUNT'].values

# Split data into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

class CrimeLikelihoodNeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(CrimeLikelihoodNeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, 32)
        self.bn4 = nn.BatchNorm1d(32)
        self.fc5 = nn.Linear(32, 16)
        self.bn5 = nn.BatchNorm1d(16)
        self.fc6 = nn.Linear(16, 1) 
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = torch.relu(self.bn5(self.fc5(x)))
        x = self.fc6(x)
        return x

# Initialize model
input_dim = X_train.shape[1]
model = CrimeLikelihoodNeuralNetwork(input_dim)

# Check if MPS (MacBook Metal GPU support) is available
device = torch.device("mps" if torch.backends.mps.is_built() else "cpu")

# Moving the model and data to the new device if it is enabled
model.to(device)
X_train_tensor = X_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error Loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
number_epochs = 100
for epoch in range(number_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Print loss every 10 epochs
    if epoch % 10 == 0:
        print(f'Epoch [{epoch+1}/{number_epochs}], Loss: {loss.item():.4f}')

# Evaluating the model
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)
    
    # Move the tensor to CPU before converting to NumPy array
    y_pred = y_pred_tensor.cpu().numpy()

    # Calculate RMSE, MAE, and R2
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Print key metrics
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"R2 Score: {r2:.4f}")

Epoch [1/100], Loss: 2.0980
Epoch [11/100], Loss: 0.2976
Epoch [21/100], Loss: 0.1432
Epoch [31/100], Loss: 0.1298
Epoch [41/100], Loss: 0.1266
Epoch [51/100], Loss: 0.1257
Epoch [61/100], Loss: 0.1254
Epoch [71/100], Loss: 0.1252
Epoch [81/100], Loss: 0.1251
Epoch [91/100], Loss: 0.1250
Mean Squared Error (MSE): 0.1269
Root Mean Squared Error (RMSE): 0.3562
Mean Absolute Error (MAE): 0.1714
R2 Score: -0.0015


### Other Neural Networks
This section is not important, I simply copied a few different neural networks that I attempted to use. These are difficult to tune and I don't think that my data is helping either. I tried others beyond what I have pasted in this section for posterity.

```python
class CrimeLikelihoodNeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(CrimeLikelihoodNeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 16)
        self.fc6 = nn.Linear(16, 1) 

        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.fc6(x)
        return x
```

* Mean Squared Error (MSE): 0.5807
* Root Mean Squared Error (RMSE): 0.7621
* Mean Absolute Error (MAE): 0.6745
* R2 Score: -3.6087
```python
class CrimeLikelihoodNeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(CrimeLikelihoodNeuralNetwork, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 64)
        self.fc7 = nn.Linear(64, 32)
        self.fc8 = nn.Linear(32, 16)
        self.fc9 = nn.Linear(16, 8)
        self.fc10 = nn.Linear(8, 1)
        
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.dropout(x)
        x = torch.relu(self.fc4(x))
        x = self.dropout(x)
        x = torch.relu(self.fc5(x))
        x = self.dropout(x)
        x = torch.relu(self.fc6(x))
        x = self.dropout(x)
        x = torch.relu(self.fc7(x))
        x = self.dropout(x)
        x = torch.relu(self.fc8(x))
        x = self.dropout(x)
        x = torch.relu(self.fc9(x))
        x = self.fc10(x)
        return x
```

```python
class CrimeLikelihoodNeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(CrimeLikelihoodNeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, 32)
        self.bn4 = nn.BatchNorm1d(32)
        self.fc5 = nn.Linear(32, 16)
        self.bn5 = nn.BatchNorm1d(16)
        self.fc6 = nn.Linear(16, 1)

        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = torch.relu(self.bn5(self.fc5(x)))
        x = self.fc6(x)
        return x
```

### Conclusions Regarding Neural Networks
The neural network showed decent results with 350,000 data points but struggled with high variance and overfitting. While the model improved over time, its performance didn't meet expectations in terms of generalization. The complexity of the data may have been better suited for a different model. Given the task and dataset size, simpler models like random forests could be more effective. Moving forward, testing a random forest might provide better results.

## Exploring Alternatives: Random Forest


### Crime Likelihood Model (RandomForestRegressor)

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Get a copy of the initial data frame
df = original_df.copy()

# Group by grid cell, day of year, and hour to count crimes
crime_counts = df.groupby(['X_BLOCK', 'Y_BLOCK', 'DAY_OF_YEAR', 'HOUR_SIN', 'HOUR_COS']).size().reset_index(name='CRIME_COUNT')

# Features and target
X = crime_counts[['X_BLOCK', 'Y_BLOCK', 'DAY_OF_YEAR', 'HOUR_SIN', 'HOUR_COS']]
y = crime_counts['CRIME_COUNT']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predictions
y_pred = rf_model.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R2 Score: {r2}")

Mean Squared Error (MSE): 0.13450888916507858
Root Mean Squared Error (RMSE): 0.3667545353026716
Mean Absolute Error (MAE): 0.17365937525892786
R2 Score: -0.061930768235775036


### Crime Type Model (RandomForestClassifier)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, mean_squared_error
from sklearn.model_selection import train_test_split

# Get a copy of the initial data frame
df = original_df.copy()

# Encode the target variable
encoder = LabelEncoder()
df['OFFENSE_TYPE_ID'] = encoder.fit_transform(df['OFFENSE_TYPE_ID'])

# Features and target
X = df.drop('OFFENSE_TYPE_ID', axis=1)
y = df['OFFENSE_TYPE_ID']

# Remove rare classes with only 1 instance
class_counts = y.value_counts()
rare_classes = class_counts[class_counts < 2].index

# Filter out the rare classes from the dataset
filtered_df = df[~df['OFFENSE_TYPE_ID'].isin(rare_classes)]

# Update X and y after removing rare classes
X = filtered_df.drop('OFFENSE_TYPE_ID', axis=1)
y = filtered_df['OFFENSE_TYPE_ID']

# Train/test split with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Ensure no class is left with fewer than 2 members in either train or test
train_class_counts = y_train.value_counts()
test_class_counts = y_test.value_counts()

# Print out the counts for train and test to debug
print("Train Class Distribution:\n", train_class_counts)
print("Test Class Distribution:\n", test_class_counts)

# Initialize and train the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predictions
y_pred = rf_model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Classification report (includes precision, recall, F1-score)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Precision, Recall, and F1 Score
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
print(f"Precision (weighted): {precision}")
print(f"Recall (weighted): {recall}")
print(f"F1 Score (weighted): {f1}")

Train Class Distribution:
 OFFENSE_TYPE_ID
150    47402
148    32539
35     22177
154    20602
153    18918
       ...  
92         2
177        2
68         2
90         2
86         2
Name: count, Length: 171, dtype: int64
Test Class Distribution:
 OFFENSE_TYPE_ID
150    11851
148     8135
35      5544
154     5151
153     4730
       ...  
113        1
86         1
123        1
109        1
94         1
Name: count, Length: 164, dtype: int64
Accuracy: 0.1844433739827092

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.05      0.03      0.04        61
           2       0.02      0.01      0.01       187
           3       0.06      0.04      0.04      1442
           4       0.01      0.00      0.00       656
           6       0.07      0.04      0.05        28
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00        26
          

/Users/kzaremski/Library/Mobile Documents/com~apple~CloudDocs/School/CS 3120 001 - Machine Learning/CS 3120 Project/denver-crime-heatmap/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kzaremski/Library/Mobile Documents/com~apple~CloudDocs/School/CS 3120 001 - Machine Learning/CS 3120 Project/denver-crime-heatmap/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kzaremski/Library/Mobile Documents/com~apple~CloudDocs/School/CS 3120 001 - Machine Learning/CS 3120 Pr

### Random Forest Conclusions
The Random Forest model did not show significant improvement compared to the Gradient Boosting or Neural Network models. Despite being a powerful ensemble method, its performance was limited, as evidenced by the relatively low precision, recall, and F1 score. These results indicate that while Random Forest can be effective in some cases, it did not provide the expected boost in predictive power over the other models tested. Further tuning or a different approach may be necessary to improve performance.

## Fine Tuning Gradient Boosting Models
Since we entertained some of the best performance on the Gradient Boosted models, I am moving forward with using them as my final models to make predictions in the production environment.

In [15]:
# References to be assigned in the future once we have identified the models that we want to pickle
classifier_model = None
regressor_model = None

### Fine Tuning the Classifier Model

In [16]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, mean_squared_error

# Get a copy of the initial data frame
df = original_df.copy()

# Encode the target variable
encoder = LabelEncoder()
df['OFFENSE_TYPE_ID'] = encoder.fit_transform(df['OFFENSE_TYPE_ID'])

# Features and target
X = df.drop('OFFENSE_TYPE_ID', axis=1)
y = df['OFFENSE_TYPE_ID']

# Remove rare classes with only less tha 16 instances
class_counts = y.value_counts()
rare_classes = class_counts[class_counts < 16].index

# Filter out the rare classes from the dataset
filtered_df = df[~df['OFFENSE_TYPE_ID'].isin(rare_classes)]

# Update X and y after removing rare classes
X = filtered_df.drop('OFFENSE_TYPE_ID', axis=1)
y = filtered_df['OFFENSE_TYPE_ID']

# Train/test split with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Ensure no class is left with fewer than 2 members in either train or test
train_class_counts = y_train.value_counts()
test_class_counts = y_test.value_counts()

# Print out the counts for train and test to debug
print("Train Class Distribution:\n", train_class_counts)
print("Test Class Distribution:\n", test_class_counts)

# Randomized search parameters
param_distributions = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_iter': [100, 200, 300, 500],
    'max_depth': [None, 3, 5, 7],
    'l2_regularization': [0, 0.01, 0.1, 1.0],
    'min_samples_leaf': [10, 20, 30, 50],
}

# Initialize the model
model = HistGradientBoostingClassifier()

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=10,
    cv=5,       # 5-fold cross-validation
    scoring='f1_weighted',  # Optimize for weighted F1-score
    random_state=42,
    n_jobs=-1   # Use all available cores
)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Best parameters
print("\nBest Parameters:", random_search.best_params_)

# Train the final model with best parameters
best_model = random_search.best_estimator_

# Predictions
y_pred = best_model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Classification report (includes precision, recall, F1-score)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Precision, Recall, and F1 Score1
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
print(f"Precision (weighted): {precision}")
print(f"Recall (weighted): {recall}")
print(f"F1 Score (weighted): {f1}")

# RMSE (Root Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# To decode the predictions back to the original labels
decoded_predictions = encoder.inverse_transform(y_pred)

# Save the best classifier model
classifier_model = best_model

Train Class Distribution:
 OFFENSE_TYPE_ID
150    47402
148    32539
35     22177
154    20602
153    18918
       ...  
46        18
69        17
110       16
10        15
70        15
Name: count, Length: 149, dtype: int64
Test Class Distribution:
 OFFENSE_TYPE_ID
150    11851
148     8135
35      5544
154     5151
153     4730
       ...  
69         4
110        4
10         4
70         4
46         4
Name: count, Length: 149, dtype: int64

Best Parameters: {'min_samples_leaf': 30, 'max_iter': 500, 'max_depth': None, 'learning_rate': 0.01, 'l2_regularization': 0.01}
Accuracy: 0.22740125530970645

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.23      0.05      0.08        61
           2       0.00      0.00      0.00       187
           3       0.11      0.01      0.01      1442
           4       0.00      0.00      0.00       656
           6       0.00      0.00      0.0

/Users/kzaremski/Library/Mobile Documents/com~apple~CloudDocs/School/CS 3120 001 - Machine Learning/CS 3120 Project/denver-crime-heatmap/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kzaremski/Library/Mobile Documents/com~apple~CloudDocs/School/CS 3120 001 - Machine Learning/CS 3120 Project/denver-crime-heatmap/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kzaremski/Library/Mobile Documents/com~apple~CloudDocs/School/CS 3120 001 - Machine Learning/CS

### Fine Tuning the Regressor Model

In [17]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Get a copy of the initial data frame
df = original_df.copy()

# Group by grid cell, day of year, and hour to count crimes
crime_counts = df.groupby(['X_BLOCK', 'Y_BLOCK', 'DAY_OF_YEAR', 'HOUR_SIN', 'HOUR_COS']).size().reset_index(name='CRIME_COUNT')

# Features and target
X = crime_counts[['X_BLOCK', 'Y_BLOCK', 'DAY_OF_YEAR', 'HOUR_SIN', 'HOUR_COS']]
y = crime_counts['CRIME_COUNT']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid for GridSearchCV
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_iter': [100, 200, 300],
    'max_depth': [7],
    'l2_regularization': [0, 0.01, 0.1],
    'min_samples_leaf': [10, 20, 30]
}

# Initialize the base model
model = HistGradientBoostingRegressor()

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Optimize for negative MSE
    cv=5,  # 5-fold cross-validation
    verbose=1,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
print("\nBest Parameters:", best_params)

best_model = grid_search.best_estimator_

# Predictions using the best model
y_pred = best_model.predict(X_test)

# Evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print metrics
print(f"\nEvaluation Metrics:")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R2 Score: {r2}")

# Save the best regressor model
regressor_model = best_model

Fitting 5 folds for each of 108 candidates, totalling 540 fits

Best Parameters: {'l2_regularization': 0.1, 'learning_rate': 0.2, 'max_depth': 7, 'max_iter': 100, 'min_samples_leaf': 10}

Evaluation Metrics:
Mean Squared Error (MSE): 0.12060748392795712
Root Mean Squared Error (RMSE): 0.3472858821316483
Mean Absolute Error (MAE): 0.15950891086751307
R2 Score: 0.047819078295898376


### Final Conclusions & Accuracies
I didn’t manage to create an accurate model by any stretch, but I’ve made progress in getting something functional. The models are far from perfect, with high error rates and low predictive power, but they do provide a starting point. The goal was to get to a point where I can move forward with building the app, and that’s where things stand now. The next step is to get the app up and running, with the understanding that these models will need serious refinement down the road. For now, the focus is on deployment and improving the models as I continue.

#### Regressor (Crime Likelihood)
```
Mean Squared Error (MSE): 0.12060748392795712
Root Mean Squared Error (RMSE): 0.3472858821316483
Mean Absolute Error (MAE): 0.15950891086751307
R2 Score: 0.047819078295898376
```
The regressor model isn’t performing great with a low R2 score, meaning it’s not explaining much of the variance in crime likelihood. The MSE is okay, but the RMSE shows that the predictions are a bit off from the actual values. The MAE tells us that, on average, the model is off by about 0.16. With the R2 score being low, there’s definitely room for improvement in future iterations. We’ll need better features or model tweaks to improve accuracy.


#### Classifier (Crime Type)
```
Precision (weighted): 0.19163135273035
Recall (weighted): 0.22740125530970645
F1 Score (weighted): 0.16102398498675685
Root Mean Squared Error (RMSE): 71.35365588711969
```

The classifier is struggling with low precision and recall, which means it’s not accurately identifying crime types. The F1 score reflects this imbalance, showing room for improvement. The RMSE also shows there’s quite a bit of error in the predictions. This model definitely needs some work to get more reliable results. For now, it gets the job done, but it’s far from optimal.

## Saving Models and Related Data to Files
For use in the production environment. In this section, we're saving our trained models and encoders using `pickle` so they can be easily loaded into the Flask web app later. By serializing the `likelihood_regressor_model`, `type_classifier_model`, and `type_classifier_encoder` into files, we ensure that the models are ready for deployment. These files are stored in the `artifacts` folder, making it simple to load them into the Flask app when it’s running. This way, we can make predictions in real-time without needing to retrain the models.

In [19]:
import pickle

with open('artifacts/likelihood_regressor_model.pkl', 'wb') as file:
    pickle.dump(regressor_model, file)

with open('artifacts/type_classifier_model.pkl', 'wb') as file:
    pickle.dump(classifier_model, file)

with open('artifacts/type_classifier_encoder.pkl', 'wb') as file:
    pickle.dump(encoder, file)

# Other Models

In [3]:
from IPython.display import display
import pandas as pd
import math
import pyproj
import numpy as np
import glob

# Find all crime data CSV files matching the naming pattern
csv_files = glob.glob("data/crime_split_*.csv")

# Read in all of the files and create Pandas dataframes from them
split_dfs = [pd.read_csv(f) for f in csv_files]

# Concatenate all the dataframes into a single one
df = pd.concat(split_dfs, ignore_index=True)

display(df.head())

original_dataframe_shape = df.shape
print(f'Loaded {original_dataframe_shape[0]} records!')

,OBJECTID,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,...,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,VICTIM_COUNT,x,y
0,20000,2020467360,2020467360299901,2999,1,criminal-mischief-mtr-veh,public-disorder,8/2/2020 10:43:00 PM,NaN,8/2/2020 10:43:00 PM,...,-105.024597,39.689751,4,422,ruby-hill,1,0,1,3.133789e+06,1.676471e+06
1,20001,20196003434,20196003434299901,2999,1,criminal-mischief-mtr-veh,public-disorder,4/20/2019 7:30:00 AM,4/20/2019 8:45:00 AM,4/20/2019 6:30:00 PM,...,-104.987348,39.714316,3,311,speer,1,0,1,3.144221e+06,1.685476e+06
2,20002,2020123049,2020123049299901,2999,1,criminal-mischief-mtr-veh,public-disorder,2/25/2020 11:30:00 PM,2/26/2020 6:45:00 AM,2/26/2020 7:30:00 AM,...,-104.988098,39.764528,6,612,five-points,1,0,1,3.143907e+06,1.703765e+06
3,20003,2021621685,2021621685299901,2999,1,criminal-mischief-mtr-veh,public-disorder,11/1/2021 6:20:00 AM,11/1/2021 6:30:00 AM,11/1/2021 10:15:00 AM,...,-105.004665,39.739669,1,123,lincoln-park,1,0,1,3.139299e+06,1.694684e+06
4,20004,2020289138,2020289138299901,2999,1,criminal-mischief-mtr-veh,public-disorder,5/9/2020 12:00:00 PM,NaN,5/11/2020 12:05:00 PM,...,-104.830661,39.795281,5,521,montbello,1,0,1,3.188083e+06,1.715255e+06


Loaded 394736 records!


In [9]:
print('OFFENSE_CATEGORY_ID:\n * ' + '\n * '.join(df['OFFENSE_CATEGORY_ID'].unique()))
print()
print('OFFENSE_TYPE_ID:\n * ' + '\n * '.join(df['OFFENSE_TYPE_ID'].unique()))

OFFENSE_CATEGORY_ID:
 * public-disorder
 * drug-alcohol
 * theft-from-motor-vehicle
 * larceny
 * other-crimes-against-persons
 * all-other-crimes
 * murder
 * robbery
 * aggravated-assault
 * arson
 * burglary
 * auto-theft
 * white-collar-crime

OFFENSE_TYPE_ID:
 * criminal-mischief-mtr-veh
 * criminal-mischief-graffiti
 * drug-hallucinogen-mfr
 * drug-hallucinogen-sell
 * drug-hallucinogen-possess
 * drug-heroin-sell
 * drug-heroin-possess
 * theft-items-from-vehicle
 * burglary-vending-machine
 * theft-from-bldg
 * drug-opium-or-deriv-sell
 * drug-opium-or-deriv-possess
 * drug-cocaine-sell
 * drug-cocaine-possess
 * drug-synth-narcotic-sell
 * drug-synth-narcotic-possess
 * drug-poss-paraphernalia
 * drug-marijuana-sell
 * drug-marijuana-possess
 * drug-marijuana-cultivation
 * drug-methamphetamine-mfr
 * drug-methampetamine-sell
 * drug-methampetamine-possess
 * drug-barbiturate-mfr
 * drug-barbiturate-sell
 * drug-barbiturate-possess
 * drug-pcs-other-drug
 * drug-make-sell-othe

What if the issue was too many categories?

In [11]:
# Keep only the columns that we want
df = df[['OFFENSE_CATEGORY_ID','FIRST_OCCURRENCE_DATE','GEO_LON','GEO_LAT']]

# Displaying a random sample of the data to confirm what it looks like
display(df.sample(n=20))

,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,GEO_LON,GEO_LAT
318029,larceny,7/26/2020 3:00:00 AM,-104.980216,39.743887
391216,aggravated-assault,3/10/2023 4:00:00 PM,-105.061707,39.654226
15606,theft-from-motor-vehicle,2/6/2021 4:10:00 PM,-105.029968,39.700087
62207,public-disorder,9/20/2019 10:15:00 PM,-104.941432,39.708120
296025,theft-from-motor-vehicle,8/20/2023 7:00:00 PM,-104.946670,39.665493
149324,burglary,8/5/2022 12:00:00 AM,-104.909416,39.775094
125614,larceny,9/14/2024 4:00:00 PM,-104.982543,39.726770
325728,larceny,7/11/2021 7:30:00 AM,-104.903154,39.654000
68472,public-disorder,7/7/2021 7:45:00 PM,-105.049997,39.741312
237259,other-crimes-against-persons,11/6/2020 11:03:00 AM,-104.977850,39.744989


In [12]:
# Convert FIRST_OCCURRENCE_DATE to Pandas date time format
df['FIRST_OCCURRENCE_DATE'] = pd.to_datetime(df['FIRST_OCCURRENCE_DATE'], format='%m/%d/%Y %I:%M:%S %p')

# Extract date and time components
df['YEAR'] = df['FIRST_OCCURRENCE_DATE'].dt.year
df['DAY_OF_WEEK'] = df['FIRST_OCCURRENCE_DATE'].dt.dayofweek  # 0 = Monday, 6 = Sunday
df['DAY_OF_YEAR'] = df['FIRST_OCCURRENCE_DATE'].dt.dayofyear
df['HOUR_COS'] = np.cos(2 * np.pi * df['FIRST_OCCURRENCE_DATE'].dt.hour / 24) # Learned that some models interpret hour 0 and 
df['HOUR_SIN'] = np.sin(2 * np.pi * df['FIRST_OCCURRENCE_DATE'].dt.hour / 24) #   hour 23 as far apart even though they are together. Sin/Cos make continuous.

# Finally, dropping the FIRST_OCCURRENCE_DATE column
df.drop(columns=['FIRST_OCCURRENCE_DATE'], inplace=True)

# Displaying a random sample of the data to confirm what it looks like
display(df.sample(n=5))

,OFFENSE_CATEGORY_ID,GEO_LON,GEO_LAT,YEAR,DAY_OF_WEEK,DAY_OF_YEAR,HOUR_COS,HOUR_SIN
375083,drug-alcohol,-105.055325,39.734441,2023,4,335,0.258819,-0.965926
296432,auto-theft,-104.983265,39.737353,2023,0,233,-0.258819,-0.965926
382390,public-disorder,-104.936108,39.747067,2023,3,355,-0.707107,-0.707107
181253,larceny,-104.903129,39.720011,2020,2,281,-0.258819,-0.965926
73633,public-disorder,-104.973226,39.727497,2022,2,236,-0.707107,-0.707107


In [13]:
GRID_SIZE = 50

# Initialize UTM projection (assuming UTM Zone 13N for Denver)
#   https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system
#   https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system#/media/File:Universal_Transverse_Mercator_zones.svg
utm_proj = pyproj.Proj(proj="utm", zone=13, datum="WGS84")

def get_grid_block(lat, lon):
    try:
        # Convert latitude and longitude to UTM coordinates (X, Y)
        x, y = utm_proj(lon, lat)  # Correct order: lon, lat
        
        # Calculate the X and Y blocks
        x_block = int(x // GRID_SIZE)
        y_block = int(y // GRID_SIZE)
        
        return x_block, y_block
    except:
        # If there is an issue parsing, return -1
        return -1, -1

# Apply the function to the DataFrame
df[['X_BLOCK', 'Y_BLOCK']] = df.apply(lambda row: get_grid_block(row['GEO_LAT'], row['GEO_LON']), axis=1, result_type='expand')

# Drop rows where X_BLOCK or Y_BLOCK is -1
df = df[(df['X_BLOCK'] != -1) & (df['Y_BLOCK'] != -1)]

# Note change in size
post_null_geo_block_removal_dataframe_shape = df.shape
print(f'Dataframe size before removal: {original_dataframe_shape[0]} items')
print(f'Dataframe size after removal: {post_null_geo_block_removal_dataframe_shape[0]} items')
print(f'--> {original_dataframe_shape[0] - post_null_geo_block_removal_dataframe_shape[0]} items removed!')

# Finally, dropping the GEO_LAT and GEO_LON columns since we have replaced them with X_BLOCK and Y_BLOCK
df.drop(columns=['GEO_LAT'], inplace=True)
df.drop(columns=['GEO_LON'], inplace=True)

# Displaying a random sample of the data to confirm what it looks like
display(df.sample(n=5))

df_shape_before_location_outlier_drop = df.shape

Q1 = df['X_BLOCK'].quantile(0.01)
Q3 = df['X_BLOCK'].quantile(0.99)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df.loc[(df['X_BLOCK'] >= lower_bound) & (df['X_BLOCK'] <= upper_bound)]

# Calculate and display rows removed
df_shape_after_location_outlier_drop = df.shape
print(f'Size before location outlier removal: {df_shape_before_location_outlier_drop[0]} records.')
print(f'Size after location outlier removal: {df_shape_after_location_outlier_drop[0]} records.')
print(f'--> {df_shape_before_location_outlier_drop[0] - df_shape_after_location_outlier_drop[0]} records removed!')

Dataframe size before removal: 394736 items
Dataframe size after removal: 394475 items
--> 261 items removed!


,OFFENSE_CATEGORY_ID,YEAR,DAY_OF_WEEK,DAY_OF_YEAR,HOUR_COS,HOUR_SIN,X_BLOCK,Y_BLOCK
389490,murder,2023,3,61,0.965926,0.258819,10236,87797
108706,burglary,2021,6,171,0.965926,-0.258819,10106,87961
32753,drug-alcohol,2019,1,281,-0.965926,0.258819,10019,87859
33987,drug-alcohol,2021,3,308,-0.866025,0.500000,9995,88011
332303,larceny,2020,3,247,-0.965926,-0.258819,10055,87993


Size before location outlier removal: 394475 records.
Size after location outlier removal: 394434 records.
--> 41 records removed!


In [16]:
# Reorder columns
df = df[[
    'OFFENSE_CATEGORY_ID',
    'X_BLOCK',
    'Y_BLOCK',
    'YEAR',
    'DAY_OF_YEAR',
    'DAY_OF_WEEK',
    'HOUR_SIN',
    'HOUR_COS',
]]
display(df.sample(n=5))
original_df = df.copy()

,OFFENSE_CATEGORY_ID,X_BLOCK,Y_BLOCK,YEAR,DAY_OF_YEAR,DAY_OF_WEEK,HOUR_SIN,HOUR_COS
50560,theft-from-motor-vehicle,9947,87864,2019,360,3,-0.500000,8.660254e-01
83930,theft-from-motor-vehicle,10035,87954,2019,319,4,0.258819,9.659258e-01
289068,aggravated-assault,10167,87979,2022,286,3,-0.965926,-2.588190e-01
303506,larceny,10185,88069,2023,275,0,-1.000000,-1.836970e-16
278797,all-other-crimes,10060,87979,2024,101,2,-0.965926,-2.588190e-01


In [17]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Get a copy of the initial data frame
df = original_df.copy()

# Group by grid cell, day of year, and hour to count crimes
crime_counts = df.groupby(['X_BLOCK', 'Y_BLOCK', 'YEAR', 'DAY_OF_YEAR', 'DAY_OF_WEEK', 'HOUR_SIN', 'HOUR_COS']).size().reset_index(name='CRIME_COUNT')

# Features and target
X = crime_counts[['X_BLOCK', 'Y_BLOCK', 'YEAR', 'DAY_OF_YEAR', 'DAY_OF_WEEK', 'HOUR_SIN', 'HOUR_COS']]
y = crime_counts['CRIME_COUNT']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid for GridSearchCV
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_iter': [100, 200, 300],
    'max_depth': [7],
    'l2_regularization': [0, 0.01, 0.1],
    'min_samples_leaf': [10, 20, 30]
}

# Initialize the base model
model = HistGradientBoostingRegressor()

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Optimize for negative MSE
    cv=5,  # 5-fold cross-validation
    verbose=1,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
print("\nBest Parameters:", best_params)

best_model = grid_search.best_estimator_

# Predictions using the best model
y_pred = best_model.predict(X_test)

# Evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print metrics
print(f"\nEvaluation Metrics:")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R2 Score: {r2}")

# Save the best regressor model
regressor_model = best_model

Fitting 5 folds for each of 108 candidates, totalling 540 fits

Best Parameters: {'l2_regularization': 0, 'learning_rate': 0.1, 'max_depth': 7, 'max_iter': 200, 'min_samples_leaf': 20}

Evaluation Metrics:
Mean Squared Error (MSE): 0.10421870655349885
Root Mean Squared Error (RMSE): 0.32282922196340724
Mean Absolute Error (MAE): 0.14820025331663794
R2 Score: 0.030293926173318053


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, mean_squared_error

# Get a copy of the initial data frame
df = original_df.copy()

# Encode the target variable
encoder = LabelEncoder()
df['OFFENSE_CATEGORY_ID'] = encoder.fit_transform(df['OFFENSE_CATEGORY_ID'])

# Features and target
X = df.drop('OFFENSE_CATEGORY_ID', axis=1)
y = df['OFFENSE_CATEGORY_ID']

# Remove rare classes with only 1 instance
class_counts = y.value_counts()
rare_classes = class_counts[class_counts < 2].index

# Filter out the rare classes from the dataset
filtered_df = df[~df['OFFENSE_CATEGORY_ID'].isin(rare_classes)]

# Update X and y after removing rare classes
X = filtered_df.drop('OFFENSE_CATEGORY_ID', axis=1)
y = filtered_df['OFFENSE_CATEGORY_ID']

# Train/test split with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Ensure no class is left with fewer than 2 members in either train or test
train_class_counts = y_train.value_counts()
test_class_counts = y_test.value_counts()

# Print out the counts for train and test to debug
print("Train Class Distribution:\n", train_class_counts)
print("Test Class Distribution:\n", test_class_counts)

# Check if any class is left with 1 sample in the train set
if any(train_class_counts < 2):
    print("Some classes in the train set have less than 2 samples. Consider further reducing or combining classes.")
else:
    # Initialize and train the model
    model = HistGradientBoostingClassifier()
    model.fit(X_train, y_train)

    # Predictions
    y_pred = model.predict(X_test)

    # Accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    # Classification report (includes precision, recall, F1-score)
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

    # Precision, Recall, and F1 Score
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    print(f"Precision (weighted): {precision}")
    print(f"Recall (weighted): {recall}")
    print(f"F1 Score (weighted): {f1}")

    # RMSE (Root Mean Squared Error)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print(f"Root Mean Squared Error (RMSE): {rmse}")

    # To decode the predictions back to the original labels
    decoded_predictions = encoder.inverse_transform(y_pred)

    # Save the best classifier model
    classifier_model = model

Train Class Distribution:
 OFFENSE_CATEGORY_ID
11    53142
3     48311
9     47696
6     46435
1     37364
4     22891
8     16838
5     16142
0     14663
10     5645
12     5403
2       682
7       335
Name: count, dtype: int64
Test Class Distribution:
 OFFENSE_CATEGORY_ID
11    13285
3     12078
9     11924
6     11609
1      9341
4      5723
8      4209
5      4036
0      3666
10     1411
12     1351
2       170
7        84
Name: count, dtype: int64
Accuracy: 0.28638432187812946

Classification Report:
               precision    recall  f1-score   support

           0       0.18      0.00      0.01      3666
           1       0.30      0.36      0.33      9341
           2       0.02      0.03      0.02       170
           3       0.28      0.41      0.33     12078
           4       0.36      0.09      0.15      5723
           5       0.44      0.35      0.39      4036
           6       0.34      0.40      0.36     11609
           7       0.00      0.00      0.00        84
 

#### No real improvement with changing of the grid size... abandoning.

In [20]:
import pickle

with open('artifacts/50m_crime_count_model.pkl', 'wb') as file:
    pickle.dump(regressor_model, file)

with open('artifacts/50m_crime_type_model.pkl', 'wb') as file:
    pickle.dump(classifier_model, file)

with open('artifacts/50m_crime_type_model_encoder.pkl', 'wb') as file:
    pickle.dump(encoder, file)